In [192]:
import cv2
import numpy as np
import sklearn.cluster

class camera_Data:
    def __init__(self, name, cam_R, cam_T, flag):
        self.name = name
        self.R = cam_R
        self.T = cam_T
        self.flag = flag

        
def cluster_pose(cam_R, cam_T):
    """
    :type ls_T: list[Transform]
    """
    # cluster t
    print(len(cam_T))
    meanshift_t = sklearn.cluster.MeanShift(bandwidth=1000, bin_seeding=True)
    meanshift_t.fit(cam_T)
    print(meanshift_t.labels_)
    #assert np.count_nonzero(meanshift_t.labels_ == 0) > (0.7 * len(cam_T))
    t = meanshift_t.cluster_centers_[0]
    print(t)

    # cluster R
    _Rs = np.array(cam_R)[meanshift_t.labels_ == 0]
    _Rs = _Rs.reshape((-1, 9))
    meanshift_R = sklearn.cluster.MeanShift(bandwidth=0.1)
    meanshift_R.fit(_Rs)
    print(meanshift_R.labels_)
    _tmp = meanshift_t.labels_ == 0
    assert isinstance(_tmp, np.ndarray)
    assert np.count_nonzero(_tmp) > (0.7 * len(_Rs))
    R = meanshift_R.cluster_centers_[0].reshape((3, 3))

    # normalize
    R = cv2.Rodrigues(cv2.Rodrigues(R)[0])[0]
    return R, t

def RT_calculate(cam1, cam2):
    f = cam1.flag & cam2.flag
    r1 = cam1.R[f]
    t1 = cam1.T[f]
    r2 = cam2.R[f]
    t2 = cam2.T[f]
    R = []
    T = []
    for i in range(r1.shape[0]):
        r12 = np.dot(cv2.Rodrigues(r1[i])[0], cv2.Rodrigues(r2[i])[0].T)
        #t12 = (-np.dot(cv2.Rodrigues(r2[i])[0].T, t2[i].reshape(-1))).reshape(-1)
        #t12 = np.dot(cv2.Rodrigues(r1[i])[0], t12 * s) + t1[i].reshape(-1)
        t12 = t1[i] - np.dot(r12, t2[i])
        R.append(r12)
        T.append(t12)
    R = np.squeeze(np.asarray(R))
    T = np.squeeze(np.asarray(T))

    return cluster_pose(R, T)
    

In [194]:
dir = r"C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration\1024"
cam_list = ['kinect_v2_1', 'azure_kinect_1','azure_kinect_0',  'azure_kinect_2', 'kinect_v2_2']
start_idx = [0, 0, 0, 0, 0]
n_list = [124, 124, 124, 124, 124]
col = 8
row = 11
square_size = 60.
objp = np.zeros((col*row, 3), np.float32)
objp[:, :2] = np.mgrid[0:col, 0:row].T.reshape(-1, 2) * square_size
h = 0
w = 0
corner_m = []
obj_m = []
cams = {}
for idx, cam in enumerate(cam_list):
    obj_points = []
    img_points = []
    cam_R, cam_T, flag = [], [], []
    cam_dir = "%s/%s_calib_snap" % (dir, cam)
    for i in range(n_list[idx]):
        filename = '%s/color%04i.jpg' % (cam_dir, i + start_idx[idx])
        print(i)
        img = cv2.imread(filename)
        h, w = img.shape[0], img.shape[1]
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (col, row), None)
        flag.append(ret)
        if ret:
            print(filename)
            corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), None)
            obj_points.append(objp)
            img_points.append(np.squeeze(corners2))
        else:
            obj_points.append(objp)
            img_points.append(np.zeros((col*row, 2)))    
    
    obj_points= np.array(obj_points).astype('float32')
    img_points= np.array(img_points).astype('float32')
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points[flag], img_points[flag], (w, h), None, None, flags=cv2.CALIB_RATIONAL_MODEL)
    for i in range(n_list[idx]):
        if flag[i]:
            retval, rvec, tvec = cv2.solvePnP(objectPoints=obj_points[i], imagePoints=img_points[i], cameraMatrix=mtx, distCoeffs=dist)
            cam_R.append(rvec)
            cam_T.append(tvec)
        else:
            cam_R.append(np.zeros((3,1)))
            cam_T.append(np.zeros((3,1)))
    cams[cam] = camera_Data(cam, np.asarray(cam_R), np.asarray(cam_T), np.asarray(flag))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration\1024/kinect_v2_1_calib_snap/color0100.jpg
101
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration\1024/kinect_v2_1_calib_snap/color0101.jpg
102
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration\1024/kinect_v2_1_calib_snap/color0102.jpg
103
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration\1024/kinect_v2_1_calib_snap/color0103.jpg
104
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration\1024/kinect_v2_1_calib_snap/color0104.jpg
105
C:\Users\lovettxh\Documents\GitHub\multi_camera_calibration\calibration\1024/kinect_v2_1_calib_snap/color0105.jpg
10

In [195]:
result = {}

temp_r , temp_t = RT_calculate(cams[cam_list[3]], cams[cam_list[4]])
result[cam_list[3] + "-" + cam_list[4] + "_R"] = temp_r
result[cam_list[3] + "-" + cam_list[4] + "_T"] = temp_t
temp_r , temp_t = RT_calculate(cams[cam_list[2]], cams[cam_list[3]])
result[cam_list[2] + "-" + cam_list[3] + "_R"] = temp_r
result[cam_list[2] + "-" + cam_list[3] + "_T"] = temp_t
temp_r , temp_t = RT_calculate(cams[cam_list[2]], cams[cam_list[1]])
result[cam_list[2] + "-" + cam_list[1] + "_R"] = temp_r
result[cam_list[2] + "-" + cam_list[1] + "_T"] = temp_t
temp_r , temp_t = RT_calculate(cams[cam_list[1]], cams[cam_list[0]])
result[cam_list[1] + "-" + cam_list[0] + "_R"] = temp_r
result[cam_list[1] + "-" + cam_list[0] + "_T"] = temp_t

for k, i in result.items():
    print(k)
    print(i)

18
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1755.05411089 -433.15100508 1156.9553154 ]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
42
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]
[1522.4574341   114.79700697  976.86899334]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]
39
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
[-1668.51543707   147.87110352   802.06265815]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
18
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[-1937.00554847  -244.77167949  1052.98422726]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
azure_kinect_2-kinect_v2_2_R
[[ 0.47807969  0.18101    -0.85946215]
 [-0.1261027   0.9825414   0.13678633]
 [ 0.86921684  0.04298573  0.49255894]]
azure_kinect_2-kinect_v2_2_T
[1755.05411089 -433.15100508 1156.9553154 ]
azure_kinect_0-azure_kinect_2_R
[[ 0.58549045 -0.03247262 -0.81002868]
 [-0.03981115  0.99684015 -